Based on:

@book{leborgne2022fraud,

title={Reproducible Machine Learning for Credit Card Fraud Detection - Practical Handbook},

author={Le Borgne, Yann-A{\"e}l and Siblini, Wissam and Lebichot, Bertrand and Bontempi, Gianluca},

url={https://github.com/Fraud-Detection-Handbook/fraud-detection-handbook},

year={2022},

publisher={Universit{\'e} Libre de Bruxelles}

}

Covered subchapters:
* 7.3 Autoencoders and anomaly detection

In [1]:
import datetime
import torch
import numpy as np
import wandb
import time
from sklearn.ensemble import IsolationForest
import pickle

In [2]:
%run shared_functions.py

In [3]:
%run my_shared_functions.py

In [4]:
DIR_INPUT = '../fraud-detection-handbook/simulated-data-transformed/data/'

BEGIN_DATE = "2018-06-11"
END_DATE = "2018-09-14"

print("Load  files")
%time transactions_df=read_from_files(DIR_INPUT, BEGIN_DATE, END_DATE)
print("{0} transactions loaded, containing {1} fraudulent transactions".format(len(transactions_df),transactions_df.TX_FRAUD.sum()))

output_feature="TX_FRAUD"

input_features=['TX_AMOUNT','TX_DURING_WEEKEND', 'TX_DURING_NIGHT', 'CUSTOMER_ID_NB_TX_1DAY_WINDOW',
       'CUSTOMER_ID_AVG_AMOUNT_1DAY_WINDOW', 'CUSTOMER_ID_NB_TX_7DAY_WINDOW',
       'CUSTOMER_ID_AVG_AMOUNT_7DAY_WINDOW', 'CUSTOMER_ID_NB_TX_30DAY_WINDOW',
       'CUSTOMER_ID_AVG_AMOUNT_30DAY_WINDOW', 'TERMINAL_ID_NB_TX_1DAY_WINDOW',
       'TERMINAL_ID_RISK_1DAY_WINDOW', 'TERMINAL_ID_NB_TX_7DAY_WINDOW',
       'TERMINAL_ID_RISK_7DAY_WINDOW', 'TERMINAL_ID_NB_TX_30DAY_WINDOW',
       'TERMINAL_ID_RISK_30DAY_WINDOW']

Load  files
CPU times: total: 422 ms
Wall time: 414 ms
919767 transactions loaded, containing 8195 fraudulent transactions


In [5]:
# Set the starting day for the training period, and the deltas
start_date_training = datetime.datetime.strptime("2018-07-25", "%Y-%m-%d")
delta_train=7
delta_delay=7
delta_test=7


delta_valid = delta_test

start_date_training_with_valid = start_date_training+datetime.timedelta(days=-(delta_delay+delta_valid))

(train_df, valid_df)=get_train_test_set(transactions_df,start_date_training_with_valid,
                                       delta_train=delta_train,delta_delay=delta_delay,delta_test=delta_test)

# By default, scales input data
(train_df, valid_df)=scaleData(train_df, valid_df,input_features)

##### Autoencoder

In [66]:
SEED = 42

if torch.cuda.is_available():
    DEVICE = "cuda" 
else:
    DEVICE = "cpu"
print("Selected device is",DEVICE)

seed_everything(SEED)

Selected device is cuda


In [67]:
x_train = torch.FloatTensor(train_df[input_features].values)
x_valid = torch.FloatTensor(valid_df[input_features].values)
y_train = torch.FloatTensor(train_df[output_feature].values)
y_valid = torch.FloatTensor(valid_df[output_feature].values)

In [68]:
training_set = FraudDatasetUnsupervised(x_train.to(DEVICE))
valid_set = FraudDatasetUnsupervised(x_valid.to(DEVICE))

In [69]:
training_generator,valid_generator = prepare_generators(training_set, valid_set, batch_size = 64)

In [70]:
criterion = torch.nn.MSELoss().to(DEVICE)

In [71]:
model = SimpleAutoencoder(len(input_features), 100,20).to(DEVICE)

optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)

In [72]:
config_autoencoder = dict(
    dataset_id = 'fraud-detection-handbook-transformed',
    validation = 'train test split',
    seed = 42,
    begin_date = '2018-07-25',
    delta_train = 7,
    delta_delay = 7,
    delta_test = 7,
    batch_size=64,
    num_workers=0,
    intermediate_size = 100,
    code_size = 20,
    optimizer='adam',
    lr=0.0001,
    early_stopping=True,
    early_stopping_patience=2,
    max_epochs=500,
    scale=True,
    criterion='mse'
)
wandb.init(project="mgr-anomaly-tsxai-project", config=config_autoencoder, tags=['autoencoder', 'imbalance-not-considered'])
config_autoencoder = wandb.config

In [73]:
model,training_execution_time_autoencoder,train_losses,valid_losses = training_loop_and_saving_best_wandb(model,training_generator,valid_generator,optimizer,criterion,
                                                            max_epochs=500,verbose=True, save_path='models/DL/autoencoder/simple_autoencoder_model.pt')



Epoch 0: train loss: 0.4457242426962857
valid loss: 0.11789110761766877
New best score: 0.11789110761766877

Epoch 1: train loss: 0.08445227285758031
valid loss: 0.044737996382872916
New best score: 0.044737996382872916

Epoch 2: train loss: 0.038134472722447625
valid loss: 0.028129172819803972
New best score: 0.028129172819803972

Epoch 3: train loss: 0.02394542537440847
valid loss: 0.016839713429207686
New best score: 0.016839713429207686

Epoch 4: train loss: 0.013856798302681394
valid loss: 0.009604576979687468
New best score: 0.009604576979687468

Epoch 5: train loss: 0.007683096453992661
valid loss: 0.005340768664037107
New best score: 0.005340768664037107

Epoch 6: train loss: 0.005278768580599223
valid loss: 0.004115462709533794
New best score: 0.004115462709533794

Epoch 7: train loss: 0.004053215772700612
valid loss: 0.003135705668265458
New best score: 0.003135705668265458

Epoch 8: train loss: 0.0030779137747339105
valid loss: 0.002502949561674372
New best score: 0.0025029

In [74]:
train_reconstruction = per_sample_mse_no_grad(model, training_generator)
start_time=time.time()
valid_reconstruction = per_sample_mse_no_grad(model, valid_generator)
prediction_execution_time=time.time()-start_time
print(valid_reconstruction[0:5])
print(np.mean(valid_reconstruction))

[3.35099e-05, 2.8576625e-05, 3.6815803e-05, 4.5891364e-05, 3.6327274e-05]
7.027291e-05


In [77]:
print(x_train[0])
print(model(x_train[0].to(DEVICE)))

tensor([-0.1323, -0.6306,  2.1808, -0.3003,  0.1241, -1.6917,  0.5035, -1.6630,
        -0.0482, -0.9810, -0.0816, -1.9895, -0.1231, -0.9719, -0.1436])
tensor([-0.1360, -0.6272,  2.1803, -0.2967,  0.1200, -1.6987,  0.5019, -1.6772,
        -0.0493, -0.9762, -0.0749, -1.9800, -0.1180, -0.9697, -0.1373],
       device='cuda:0', grad_fn=<AddBackward0>)


In [78]:
genuine_losses = np.array(valid_reconstruction)[y_valid.cpu().numpy() == 0]
fraud_losses = np.array(valid_reconstruction)[y_valid.cpu().numpy() == 1]
print("Average fraud reconstruction error:", np.mean(fraud_losses))
print("Average genuine reconstruction error:", np.mean(genuine_losses))

Average fraud reconstruction error: 0.0013619309
Average genuine reconstruction error: 6.196271e-05


In [79]:
wandb.log({'Training execution time': training_execution_time_autoencoder})
print(training_execution_time_autoencoder)
print(prediction_execution_time)
wandb.log({'Prediction execution time': prediction_execution_time})

282.9442689418793
1.1509997844696045


In [80]:
predictions_df=valid_df
predictions_df['predictions']=valid_reconstruction
    
performance_df = performance_assessment_f1_included(predictions_df, top_k_list=[100])
performance_df

,AUC ROC,Average precision,F1 score,Card Precision@100
0,0.84,0.163,0.0,0.2


In [81]:
wandb.log({'AUC ROC': performance_df.loc[0,'AUC ROC']})
wandb.log({'Average precision': performance_df.loc[0,'Average precision']})
wandb.log({'F1 score': performance_df.loc[0,'F1 score']})
wandb.log({'Card Precision@100': performance_df.loc[0,'Card Precision@100']})

autoenc_artifact = wandb.Artifact('autoencoder', type='autoencoder', description='trained simple autoencoder')
autoenc_artifact.add_dir('models/DL/autoencoder')
wandb.log_artifact(autoenc_artifact)
wandb.finish()

wandb: Adding directory to artifact (.\models\DL\autoencoder)... Done. 0.0s


AUC ROC,▁
Average precision,▁
Card Precision@100,▁
F1 score,▁
Prediction execution time,▁
Training execution time,▁
train loss,█▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val loss,█▄▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
AUC ROC,0.84
Average precision,0.163
Card Precision@100,0.2


##### Isolation Forest

In [78]:
config_if = dict(
    dataset_id = 'fraud-detection-handbook-transformed',
    validation = 'train test split',
    seed = 42,
    begin_date = '2018-07-25',
    delta_train = 7,
    delta_delay = 7,
    delta_test = 7,
    n_estimators = 10,
    scale=True,
)
wandb.init(project="mgr-anomaly-tsxai-project", config=config_if, tags=['isolation-forest', 'imbalance-not-considered', 'baseline'])
config_if = wandb.config

In [79]:
anomalyclassifier = IsolationForest(random_state=config_if.seed, n_estimators=config_if.n_estimators)
start_time=time.time()
anomalyclassifier.fit(train_df[input_features])
training_execution_time=time.time()-start_time
wandb.log({'Training execution time': training_execution_time})
predictions_df = valid_df
start_time=time.time()
predictions_df['predictions'] = -anomalyclassifier.score_samples(valid_df[input_features])
prediction_execution_time=time.time()-start_time
wandb.log({'Prediction execution time':  prediction_execution_time})
performance_df = performance_assessment_f1_included(predictions_df, top_k_list=[100])
performance_df

,AUC ROC,Average precision,F1 score,Card Precision@100
0,0.808,0.164,0.046,0.19


In [80]:
wandb.log({'AUC ROC': performance_df.loc[0,'AUC ROC']})
wandb.log({'Average precision': performance_df.loc[0,'Average precision']})
wandb.log({'F1 score': performance_df.loc[0,'F1 score']})
wandb.log({'Card Precision@100': performance_df.loc[0,'Card Precision@100']})
pickle.dump(anomalyclassifier, open('models/baseline/if/isolation_forest_model.sav', 'wb'))

In [81]:
classifier_artifact = wandb.Artifact('if', type='isolation_forest', description='trained baseline isolation forest')
classifier_artifact.add_dir('models/baseline/if')
wandb.log_artifact(classifier_artifact)
wandb.finish()

wandb: Adding directory to artifact (.\models\baseline\if)... Done. 0.0s


AUC ROC,▁
Average precision,▁
Card Precision@100,▁
F1 score,▁
Prediction execution time,▁
Training execution time,▁
AUC ROC,0.808
Average precision,0.164
Card Precision@100,0.19
F1 score,0.046
Prediction execution time,0.13


In [3]:
run = wandb.init()
artifact = run.use_artifact('chamera/mgr-anomaly-tsxai-project/autoencoder:v0', type='autoencoder')
artifact_dir = artifact.download()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: chamera. Use `wandb login --relogin` to force relogin


wandb:   1 of 1 files downloaded.  


##### Semi-supervised

In [50]:
seed_everything(SEED)

training_set_supervised = FraudDataset(x_train.to(DEVICE), y_train.to(DEVICE))
valid_set_supervised = FraudDataset(x_valid.to(DEVICE), y_valid.to(DEVICE))

training_generator_supervised,valid_generator_supervised = prepare_generators(training_set_supervised,
                                                                              valid_set_supervised,
                                                                              batch_size=64)

model_supervised = SimpleFraudMLPWithDropout(len(input_features), 1000, 0.2).to(DEVICE)
optimizer = torch.optim.Adam(model_supervised.parameters(), lr = 0.0001)
criterion = torch.nn.BCELoss().to(DEVICE)

model_supervised,training_execution_time,train_losses_dropout,valid_losses_dropout =\
    training_loop_eval_with_no_grad(model_supervised,
                  training_generator_supervised,
                  valid_generator_supervised,
                  optimizer,
                  criterion,
                  verbose=True)


Epoch 0: train loss: 0.10162743877167199
valid loss: 0.035735654941877054
New best score: 0.035735654941877054

Epoch 1: train loss: 0.038805231760945136
valid loss: 0.026402934476265003
New best score: 0.026402934476265003

Epoch 2: train loss: 0.03107889258012489
valid loss: 0.023788556019404057
New best score: 0.023788556019404057

Epoch 3: train loss: 0.028793505515616536
valid loss: 0.0226955978942862
New best score: 0.0226955978942862

Epoch 4: train loss: 0.02777124687149608
valid loss: 0.022126405810570862
New best score: 0.022126405810570862

Epoch 5: train loss: 0.026878768456191623
valid loss: 0.02189638417524596
New best score: 0.02189638417524596

Epoch 6: train loss: 0.02616935516404875
valid loss: 0.021739530122375797
New best score: 0.021739530122375797

Epoch 7: train loss: 0.025671390378064197
valid loss: 0.020999993936698172
New best score: 0.020999993936698172

Epoch 8: train loss: 0.02493373950921982
valid loss: 0.02112951638670689
1  iterations since best score.


In [51]:
predictions = []

for x_batch, y_batch in valid_generator_supervised: 
    predictions.append(model_supervised(x_batch.to(DEVICE)).detach().cpu().numpy())

predictions_df=valid_df
predictions_df['predictions']=np.vstack(predictions)
    
performance_assessment_f1_included(predictions_df, top_k_list=[100])

,AUC ROC,Average precision,F1 score,Card Precision@100
0,0.861,0.647,0.671,0.277


In [82]:
training_set = FraudDatasetUnsupervised(x_train.to(DEVICE))
valid_set = FraudDatasetUnsupervised(x_valid.to(DEVICE))

loader_params = {'batch_size': 64,
                 'num_workers': 0}
    
training_generator = torch.utils.data.DataLoader(training_set, **loader_params)
valid_generator = torch.utils.data.DataLoader(valid_set, **loader_params)

train_df["reconstruction_error"] = train_reconstruction
valid_df["reconstruction_error"] = valid_reconstruction

In [83]:
config_autoencoder = dict(
    dataset_id = 'fraud-detection-handbook-transformed-semisupervised',
    validation = 'train test split',
    seed = 42,
    begin_date = '2018-07-25',
    delta_train = 7,
    delta_delay = 7,
    delta_test = 7,
    batch_size=64,
    num_workers=0,
    unsupervised_intermediate_size = 100,
    unsupervised_code_size = 20,
    unsupervised_optimizer='adam',
    unsupervised_lr=0.0001,
    unsupervised_early_stopping=True,
    unsupervised_early_stopping_patience=2,
    unsupervised_max_epochs=500,
    unsupervised_scale=True,
    unsupervised_criterion='mse',
    supervised_hidden_size = 100,
    supervised_dropout=0.2,
    supervised_optimizer='adam',
    supervised_lr=0.0001,
    supervised_early_stopping=True,
    supervised_early_stopping_patience=2,
    supervised_max_epochs=500,
    supervised_scale=True,
    supervised_criterion='bce',
)
wandb.init(project="mgr-anomaly-tsxai-project", config=config_autoencoder, tags=['autoencoder', 'mlp', 'semi-supervised', 'imbalance-not-considered'])
config_autoencoder = wandb.config

In [84]:
seed_everything(SEED)

input_features_new = input_features + ["reconstruction_error"]

# Rescale the reconstruction error
(train_df, valid_df)=scaleData(train_df, valid_df, ["reconstruction_error"])

x_train_new = torch.FloatTensor(train_df[input_features_new].values)
x_valid_new = torch.FloatTensor(valid_df[input_features_new].values)

training_set_supervised_new = FraudDataset(x_train_new.to(DEVICE), y_train.to(DEVICE))
valid_set_supervised_new = FraudDataset(x_valid_new.to(DEVICE), y_valid.to(DEVICE))

training_generator_supervised,valid_generator_supervised = prepare_generators(training_set_supervised_new,
                                                                              valid_set_supervised_new,
                                                                              batch_size=64)

model_supervised = SimpleFraudMLPWithDropout(len(input_features_new), 100, 0.2).to(DEVICE)
optimizer = torch.optim.Adam(model_supervised.parameters(), lr = 0.0001)
criterion = torch.nn.BCELoss().to(DEVICE)

model_supervised,new_training_execution_time_autoencoder,train_losses_dropout,valid_losses_dropout = \
    training_loop_and_saving_best_wandb(model_supervised,
                  training_generator_supervised,
                  valid_generator_supervised,
                  optimizer,
                  criterion,
                  verbose=True,
                  save_path='models/DL/semi_sup_autoenc/semi_supervised_autoencoder_model.pt')

predictions = []
start_time=time.time()
for x_batch, y_batch in valid_generator_supervised: 
    predictions.append(model_supervised(x_batch).detach().cpu().numpy())
prediction_execution_time=time.time()-start_time



Epoch 0: train loss: 0.3238065295690674
valid loss: 0.11897474443489085
New best score: 0.11897474443489085

Epoch 1: train loss: 0.08699691516790632
valid loss: 0.05253201153923254
New best score: 0.05253201153923254

Epoch 2: train loss: 0.055241820214576236
valid loss: 0.039304085049391445
New best score: 0.039304085049391445

Epoch 3: train loss: 0.04594115533306838
valid loss: 0.0347509316652199
New best score: 0.0347509316652199

Epoch 4: train loss: 0.04037084207419541
valid loss: 0.03193916861639648
New best score: 0.03193916861639648

Epoch 5: train loss: 0.036431323328228175
valid loss: 0.030058473223677832
New best score: 0.030058473223677832

Epoch 6: train loss: 0.03379461977494628
valid loss: 0.028923461568170314
New best score: 0.028923461568170314

Epoch 7: train loss: 0.03203564086514053
valid loss: 0.02820663683375985
New best score: 0.02820663683375985

Epoch 8: train loss: 0.030814288029123105
valid loss: 0.027781046959291636
New best score: 0.027781046959291636

E

In [85]:
wandb.log({'Training execution time': training_execution_time_autoencoder + new_training_execution_time_autoencoder})
print(training_execution_time_autoencoder + new_training_execution_time_autoencoder)
print(prediction_execution_time)
wandb.log({'Prediction execution time': prediction_execution_time})

390.7052712440491
0.9259974956512451


In [86]:
predictions_df=valid_df
predictions_df['predictions']=np.vstack(predictions)
    
performance_df = performance_assessment_f1_included(predictions_df, top_k_list=[100])
performance_df

,AUC ROC,Average precision,F1 score,Card Precision@100
0,0.81,0.509,0.499,0.236


In [87]:
wandb.log({'AUC ROC': performance_df.loc[0,'AUC ROC']})
wandb.log({'Average precision': performance_df.loc[0,'Average precision']})
wandb.log({'F1 score': performance_df.loc[0,'F1 score']})
wandb.log({'Card Precision@100': performance_df.loc[0,'Card Precision@100']})

mlp_artifact = wandb.Artifact('semi_sup_autoenc', type='semisupervised', description='trained semisupervised fraud detection where autoencoder generates new input feature and mlp predicts the final score')
mlp_artifact.add_dir('models/DL/semi_sup_autoenc')
wandb.log_artifact(mlp_artifact)
wandb.finish()

wandb: Adding directory to artifact (.\models\DL\semi_sup_autoenc)... Done. 0.0s


AUC ROC,▁
Average precision,▁
Card Precision@100,▁
F1 score,▁
Prediction execution time,▁
Training execution time,▁
train loss,█▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val loss,█▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
AUC ROC,0.81
Average precision,0.509
Card Precision@100,0.236
